# **Parte 1 - Preparação dos Dados**

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0f9a39d24703ec2145caecf1926e8e9b5e97c0308da85ac52273492d08865571
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


### SparkSession: é como interagir com **Spark usando python (via API)**

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

Uma SparkSession pode ser utilizada para criar DataFrames, registrar DataFrames como tabelas, executar consultas SQL em tabelas, armazenar em cache e ler arquivos parquet. Para criar uma SparkSession, use o seguinte padrão de construtor:

<font size=2>**Fonte:** [SparkSession](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.SparkSession.html)</font>

In [2]:
from pyspark.sql import SparkSession

In [3]:
# comando master: onde estará rodando o sparksession
# appname: nome da sessão (problema que estou tentando resolver)
spark = SparkSession.builder.master('local[*]').appName("Classificação com Spark").getOrCreate()

In [4]:
spark

> **Retorno desejado da variável Spark**: versão utilizada do Spark, o ambiente e o nome que foi definido.

## **1. Carregamento dos Dados**

Utilização do método `spark.read.csv` do Spark SQL para carregar os dados em um DataFrame. Nessa etapa é importante verificar, através dos comandos `show`, `toPandas` e `count`, se os dados foram carregados da maneira correta.

**Pontos de atenção** são sempre os parâmetros header e sep, que informam qual separador é usado no arquivo e se ele conta com cabeçalho.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#import zipfile
#zipfile.ZipFile('/content/drive/MyDrive/curso-spark/base_dados/base de dados.zip', 'r').extractall('/content/drive/MyDrive/curso-spark/base_dados')
dados_path = '/content/drive/MyDrive/curso-spark/base_dados/dados_clientes.csv'
dados = spark.read.csv(dados_path, header=True, inferSchema=True)

In [ ]:
# estrutura do dataframe
dados

DataFrame[id: int, Churn: string, Mais65anos: int, Conjuge: string, Dependentes: string, MesesDeContrato: int, TelefoneFixo: string, MaisDeUmaLinhaTelefonica: string, Internet: string, SegurancaOnline: string, BackupOnline: string, SeguroDispositivo: string, SuporteTecnico: string, TVaCabo: string, StreamingFilmes: string, TipoContrato: string, ContaCorreio: string, MetodoPagamento: string, MesesCobrados: double]

In [ ]:
dados.show(5)

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|            Nao|         Sim|              Nao|           Nao|    Nao|            Nao| Mensalmente|         Sim|BoletoEletronico|       

- id: identificador do cliente;
- Churn: cancelou ou não o serviço;
- Mais65anos: variável binária, se tem é 1 e se não tem é 0;
- Conjuge: variável binbária em formato de string;
- MaisDeUmaLinhaTelefonica: mais de duas possibilidades;
- Internet: mais de uma possibilidade;
...

In [ ]:
# número de registros
dados.count()

10348

In [ ]:
# verificar se está balanceado: pessoas que cancelaram é maior, menor ou igual a quem não cancelou?
dados.select('Churn').groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [ ]:
# outra sintaxe com o mesmo efeito:
dados.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



In [ ]:
# esquema dos dados e como Spark reconheceu os tipos dos dados
# maioria está como string e isso é um problema para ML, que não irá reconhecer esse tipo
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



## **2. Transformando os Dados**

Analisando o tipo dos dados de cada coluna e também quais e quantas categorias existem para cada feature, é possível classificá-las e, assim, decidir a melhor abordagem para o preparo dos dados para os modelos de machine learning.

<font size=2>**Fonte:** [Functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions)</font>

In [7]:
# Valores com 'Sim' ou 'Não'
colunasBinarias = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [8]:
from pyspark.sql import functions as f

In [9]:
# 1: sim e 0: não > f.when(f.col(colName)=='Sim', 1).otherwise(0)
# Update no nome da coluna para que não seja a regra da primeira função > .alias(colName)
todasColunas = [f.when(f.col(colName)=='Sim', 1).otherwise(0).alias(colName) for colName in colunasBinarias]
todasColunas # todas as colunas são apenas as colunas binárias

[Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorreio = Sim) THEN 1 ELSE 0 END AS ContaCorreio'>]

In [10]:
# dados.columns tem todas as colunas dos dados, mas não tem as transformações
# quero organizar para que os dados binários fiquem no final > reversed()
for coluna in reversed(dados.columns):
  if coluna not in colunasBinarias: # objetivo: inserir as colunas que não tem na lista de colunas binárias
    todasColunas.insert(0, coluna) # insere na primeira posição

In [ ]:
todasColunas
# as primeiras colunas são as que não tiveram transformações

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorr

In [ ]:
# as regras de transformação já existem e agora precisam ser aplicadas > .select()
dados.select(todasColunas).show(10)

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

In [11]:
dataset = dados.select(todasColunas)

In [ ]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)



> Ainda existem colunas como string, no entanto elas apresentam valores não binários. Assim, um novo tratamento dos dados precisam ser executados.

In [ ]:
dataset.show(10)

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

## **3. Criando *Dummies***

Analisando as colunas que ainda são strings:

In [12]:
dataset.select(['Internet', 'TipoContrato', 'MetodoPagamento']).show(15)

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|   CartaoCredito|
|        DSL| Mensalmente|          Boleto|
|FibraOptica| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|   DebitoEmConta|
|        DSL| Mensalmente|          Boleto|
|        Nao|    DoisAnos|   CartaoCredito|
|FibraOptica|       UmAno|   CartaoCredito|
|FibraOptica| Mensalmente|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
+-----------+------------+----------------+
only showing top 15 rows



**Utilização da técnica Dummies**:

Por exemplo, no 'MetodoPagamento' existem diferentes métodos de pagamentos.

Para transformar esses dados se constrói uma coluna que represente cada uma das opções de valores para cada categoria. Isso quer dizer que para a coluna internet terá uma coluna que representa DSL, outra FibraOptica e outra referente ao Nao. Os valores dessas colunas são representadas por 0 e 1 (se o cliente tem ou não).

In [13]:
# step 1: id são únicos e a ideia de usar ele aqui é para utilizar .pivot()
# .pivot() espera receber o nome da coluna de interesse
# .pivot pega cada uma das categorias da coluna target e transforma em colunas
# step 2: como preencher as rows dessas colunas novas?
# .agg() e define como será feito esse preenchimento
# f.lit(1) vai criar uma coluna de valor 1 para as categorias que o cliente possui
# se o cliente não possui a categoria ele coloca NaN
# fora do .agg() .na vai selecionar apenas os dados que são NaN
# .fill(0) coloca 0 no lugar do NaN
# .show() para verificar o resultado
dataset.groupBy('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0).show()

+-----+------+----------------+-------------+-------------+
|   id|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+-----+------+----------------+-------------+-------------+
| 3997|     0|               0|            1|            0|
| 7554|     0|               1|            0|            0|
| 6336|     0|               1|            0|            0|
| 6357|     0|               1|            0|            0|
| 9427|     0|               0|            1|            0|
| 2659|     0|               0|            1|            0|
|  471|     0|               1|            0|            0|
| 4935|     0|               0|            1|            0|
| 4818|     0|               0|            1|            0|
| 1342|     1|               0|            0|            0|
| 1959|     0|               1|            0|            0|
| 9376|     0|               0|            1|            0|
| 2366|     0|               1|            0|            0|
| 1580|     0|               0|         

In [14]:
Internet = dataset.groupBy('id').pivot('Internet').agg(f.lit(1)).na.fill(0)
TipoContrato = dataset.groupBy('id').pivot('TipoContrato').agg(f.lit(1)).na.fill(0)
MetodoPagamento = dataset.groupBy('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0)

> Agora as colunas estão criadas e representam as informações de uma maneira diferente (dummies). O próximo passo é unir essa informação com o dataset.

In [ ]:
dataset\
    .join(Internet, 'id', how='inner')\
    .join(TipoContrato, 'id', how='inner')\
    .join(MetodoPagamento, 'id', how='inner')\
    .show(5)

+----+----------+---------------+-----------+------------+----------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+---+-----------+---+--------+-----------+-----+------+----------------+-------------+-------------+
|  id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|DSL|FibraOptica|Nao|DoisAnos|Mensalmente|UmAno|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+----+----------+---------------+-----------+------------+----------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+---+-----------+---+--------+---

> **Necessário renomar as colunas adicionar para que exibam melhor as informações que elas trazem**.

In [ ]:
# dps de .join(), .select() todas as colunas criadas e cria um novo nome
# .drop() das colunas originais
dataset\
    .join(Internet, 'id', how='inner')\
    .join(TipoContrato, 'id', how='inner')\
    .join(MetodoPagamento, 'id', how='inner')\
    .select(
        '*',
        f.col('DSL').alias('Internet_DSL'),
        f.col('FibraOptica').alias('Internet_FibraOptica'),
        f.col('Nao').alias('Internet_Nao'),
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
        f.col('UmAno').alias('TipoContrato_UmAno'),
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'),
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
        f.col('Boleto').alias('MetodoPagamento_Boleto')
    ).drop(
        'Internet', 'TipoContrato', 'MetodoPagamento', 'DSL',
        'FibraOptica', 'Nao', 'Mensalmente', 'UmAno', 'DoisAnos',
        'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
    ).show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [15]:
# sobreescreve o dataset original
dataset = dataset\
    .join(Internet, 'id', how='inner')\
    .join(TipoContrato, 'id', how='inner')\
    .join(MetodoPagamento, 'id', how='inner')\
    .select(
        '*',
        f.col('DSL').alias('Internet_DSL'),
        f.col('FibraOptica').alias('Internet_FibraOptica'),
        f.col('Nao').alias('Internet_Nao'),
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
        f.col('UmAno').alias('TipoContrato_UmAno'),
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'),
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
        f.col('Boleto').alias('MetodoPagamento_Boleto')
    ).drop(
        'Internet', 'TipoContrato', 'MetodoPagamento', 'DSL',
        'FibraOptica', 'Nao', 'Mensalmente', 'UmAno', 'DoisAnos',
        'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
    )

In [ ]:
# checa se tá certo
dataset.show(5)

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [ ]:
# verifica se as variáveis estão no formato necessário para entrar em algum modelo
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContr

### **Aplicar o melhor tratamento para cada tipo de dado:**

**Dados Categóricos**:

Através da identificação do dado se determinar quais são os melhores tratamentos de dados para serem utilizados em modelos de machine learning.

Como diferenciar dados categóricos binários e não binários:

- **Categóricos binários** são dados que só tem duas variações de valores e podem ser representados por outros valores que não o um e o zero. Por exemplo: em questionários, esses valores são registrados como _Sim_ e _Não_ ou _Verdadeiro_ e _Falso_.
- **Categóricos não binários**, também chamados de variáveis politômicas, são os que têm mais de dois valores possíveis. Nesses tipos de dados ainda temos mais uma ramificação categórica: **ordinais** e **nominais**.
  - **Ordinais**: quando os dados possuem uma ordem entre eles. Por exemplo, o grau de instrução de uma pessoa tem uma ordem: primeiro o ensino fundamental, depois o ensino médio e, finalmente, o ensino superior.
  - **Nominais**: quando os valores não têm uma ordem, como: estado civil (solteiro, casado, separado, divorciado ou viúvo).

A identificação deste tipo é muito importante para determinar como será feita a transformação do dados para números. Para dados categóricos nominais, atribuir valores para as categorias não estaria correto, já que, dessa maneira, estaria aplicando uma ordem e um valor para categorias que originalmente não tem.

No exemplo do estado civil, se atribuirmos os valores de 1, 2, 3, 4 e 5 para solteiro, casado, separado, divorciado e viúvo, respectivamente, podemos afetar o entendimento do modelo, pois podemos dizer que viúvo é maior que solteiro, já que 5 é maior que 1, e essa não é uma informação correta. Por isso, quando identificamos esse tipo de dado, o procedimento mais usado será criar dummies para essa coluna, em que cada valor de categoria terá sua própria coluna de valores binários.

# **Parte 2 - Projeto: Churn**

**Descrição e objetivo do projeto**: a partir de uma base de dados com características de diversos clientes e dos serviços contratados por esse cliente será criado um modelo que fará a classificação se um cliente que vai cancelar o serviço e o que não, se tornando um churn para a empresa.

**MLlib**: módulo do Spark que possui diferentes algoritmos de machine learning.

# Testando: **Regressão Logística**

O algoritmo de regressão logística faz parte dos algoritmos de aprendizado de máquina supervisionado e é utilizado para problemas nos quais a variável resposta é categórica, podendo ser do tipo binomial (apenas dois níveis **e é o caso desse projeto**) ou multinomial (três ou mais níveis).

Ela parece com a regressão linear, pois o algoritmo vai criar uma reta que se "ajusta" aos dados e consiga representar os dados de treino.

A diferença é que, depois que criar essa reta e ajustar aos valores de treino, aplica-se a Sigmoide que vai transformar os valores em valores entre um intervalo de 0 e 1. A partir dessa transformação, é possível separar os dados em duas classes, que no caso, é se a pessoa cancelou o serviço ou não.

A **sigmoide vai dar uma probabilidade de ser de determinada classe ou outra**; existe um valor no meio que vai separar muito bem, normalmente utilizando o valor 0.5. Então, quando atinge um valor 0.5, consider que é da classe positiva, ou seja, da primeira classe. E abaixo desse valor, considera que é da classe negativa, ou seja, a de quem cancelou o serviço, por exemplo.

## **Preparação dos Dados** - parte 2

In [ ]:
dataset.show(5)

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+------------------------+------------------+---------------------+-----------------------------+-----------------------------+--------------------------------+----------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_Mensalmente|TipoContrato_UmAno|TipoContrato_DoisAnos|MetodoPagamento_DebitoEmConta|MetodoPagamento_CartaoCredito|MetodoPagamento_BoletoEletronico|MetodoPagamento_Boleto|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

 Embora os dados sejam numéricos, o Spark precisa de um tipo de dado mais primitivo para trabalhar, então é necessário fazer uma **conversão desse DataFrame para diversos vetores que vão representar os dados**.

In [16]:
from pyspark.ml.feature import VectorAssembler

In [17]:
# Antes de usar o VectorAssembler, é preciso mudar o nome da coluna target do “Churn” pelo nome padrão esperado pelo módulo que é 'label'
dataset = dataset.withColumnRenamed('Churn', 'label') # sobreescreve o dataset

O próximo passo é obter o nome das features. Então todas as features/características que representam o problema são os dados de cliente e é preciso uma variável com uma lista contendo o nome de todas as colunas que forem features.

In [18]:
X = dataset.columns

In [19]:
# Remove a coluna 'label' e 'id' da lista com o nome das features
X = dataset.columns
X.remove('label') # não pode estar nas features, pois tem caracteristica que identifica
X.remove('id') # não agrega no modelo
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'TipoContrato_DoisAnos',
 'MetodoPagamento_DebitoEmConta',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_Boleto']

In [20]:
# dois argumentos: 1- dados de entrada do modelo que está na variável X e 2- nome da saída de dados do modelo (será um vetor único)
assembler = VectorAssembler(inputCols=X, outputCol='features')

In [21]:
# .transform() para fazer a transformação dos dados
# .select() e seleciona as colunas de interesse > colunas das features e o label (target)
dataset_prep = assembler.transform(dataset).select('features', 'label')

In [ ]:
# visualizando o vetor inteiro
dataset_prep.show(10, truncate=False)

+-----------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                   |label|
+-----------------------------------------------------------------------------------------------------------+-----+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])                               |1    |
|(24,[1,2,3,5,6,8,9,11,12,13,15,17,22],[60.0,103.6142230120257,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
|(24,[1,2,5,6,10,11,12,13,14,18,23],[12.0,75.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                       |0    |
|(24,[1,2,3,5,8,12,13,14,19,21],[69.0,61.45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                               |0    |
|(24,[1,2,3,5,6,11,13,15,17,22],[7.0,86.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                 |1    |
|(24,[1,2,5,6,12,13,15,17,22],[14.0,85.03742670311915,1.0,1.0,1.0,1.0,1.

**Analisando dataset_prep:**

> Features: o **primeiro valor**, que é o 24 (para todos), é a **quantidade de features que existem**; o **segundo é uma lista das colunas que foi armazenado um valor de feature**, que é a maneira como o vetor foi representado e apenas armazenando valores diferentes de 0 (economia de memória -> fala qual coluna ele registra e que não é 0); e o **terceiro campo é uma lista com os valores das features de fatos**.

## **Ajuste e Previsão usando modelo LR**

In [22]:
# Garante a reprodução dos resultados
SEED = 101

In [23]:
# Split em treino e teste
# lista com a % de separação dos dados 0.7 e 0.3
treino, teste = dataset_prep.randomSplit([0.7, 0.3], seed=SEED)

In [ ]:
treino.show(5, False)

+----------------------------------------------------------------------------------------------------+-----+
|features                                                                                            |label|
+----------------------------------------------------------------------------------------------------+-----+
|(24,[0,1,2,3,4,5,6,7,8,10,13,15,18,20],[1.0,58.0,89.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0    |
|(24,[0,1,2,3,4,5,6,7,8,11,14,18,22],[1.0,71.0,69.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])        |0    |
|(24,[0,1,2,3,4,5,6,7,8,12,13,15,17,22],[1.0,60.0,93.25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0    |
|(24,[0,1,2,3,4,5,6,7,8,13,15,17,22],[1.0,65.0,85.75,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |0    |
|(24,[0,1,2,3,4,5,6,7,13,15,17,22],[1.0,60.0,80.95,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])             |0    |
+----------------------------------------------------------------------------------------------------+-----+
only showing top 5 

In [ ]:
# quantidade de dados separados para o treino
treino.count()

7206

In [ ]:
teste.show(5,False)

+----------------------------------------------------------------------------------------------------+-----+
|features                                                                                            |label|
+----------------------------------------------------------------------------------------------------+-----+
|(24,[0,1,2,3,4,5,6,7,9,10,11,14,18,22],[1.0,55.0,76.25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0    |
|(24,[0,1,2,3,4,5,6,7,13,15,17,22],[1.0,24.0,79.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])             |0    |
|(24,[0,1,2,3,4,5,6,8,13,15,17,23],[1.0,34.0,79.6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])              |1    |
|(24,[0,1,2,3,4,5,6,9,11,12,13,15,17,21],[1.0,66.0,99.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
|(24,[0,1,2,3,4,5,6,9,11,13,15,17,23],[1.0,32.0,91.35,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])      |0    |
+----------------------------------------------------------------------------------------------------+-----+
only showing top 5 

In [ ]:
# quantidade de dados separados para o teste
teste.count()

3142

In [24]:
# Treinamento do Modelo
from pyspark.ml.classification import LogisticRegression

In [26]:
# lr é para logistic regression
# porém ainda não conhece os dados
lr = LogisticRegression()

In [27]:
# passando os dados de treino para o modelo par ajustar
modelo_lr = lr.fit(treino)

> Uma particularidade do Spark é que se utiliza a palavra _transform_ e não _predict_ para gerar as previsões.

In [28]:
previsoes_lr_teste = modelo_lr.transform(teste)

In [29]:
previsoes_lr_teste.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[3.02174179751551...|[0.95354674000282...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.0922192966076...|[0.47696150091605...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[0.18744121711361...|[0.54672358463156...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.91716501260103...|[0.71446410549163...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1495904711610...|[0.46267196467801...|       1.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



**As previsões possuem 5 colunas:**
- A **primeira coluna são as “Features”** que são dos dados já conhecidos, são as informações dos clientes;
- A **segunda coluna é a “Label” com o valor real**, ou seja, o histórico de cliente, se cancelou o serviço ou não;
- **'rawPrediction'** seria a'predição bruta', ou o dado bruto;
- **'probability'** é relativa à sigmoide, onde quanto mais próximo de 1, maior a probabilidade;
- **'Prediction'** é o que o modelo realmente decidiu em relação àquele dado.

Os valores de probability são equivalentes aos valores da regressão depois de passados pela função de ativação - função logística que abrange valores entre 0 e 1. O valor de predição (em prediction), nesse caso, é dado de acordo com o valor de probability de forma que:

1. Se **probability maior ou igual a 0.5, prediction será 0**; e
2. Se **probability menor que 0.5, prediction será 1**.

Os valores em rawPrediction são aqueles calculados antes de passarem pela função de ativação, ou seja, são os cálculos da regressão.

> No primeiro caso, na primeira linha, onde a label original era 0, a predição na última coluna foi 0, isso significa que o modelo acertou. no primeiro caso, por exemplo, na primeira linha, que a nossa label original era 0, e a nossa predição na última coluna foi 0, então o modelo acertou. Na segunda linha seria o segundo cliente, que não cancelou o serviço, mas o modelo previu que cancelou.

## **Métricas**

Métricas do próprio modelo pode ser acessado por `modelo_lr.summary`.

In [30]:
modelo_lr.summary

In [31]:
# salvando em uma variável
resumo_lr_treino = modelo_lr.summary

In [32]:
# métricas do treino
resumo_lr_treino.accuracy

0.7849014709963918

O retorno é um valor de aproximadamente 0.78... +casas decimais. É assim que se acessa as informações como atributos, ou seja, essas métricas do .summary em relação aos dados de treino.

In [33]:
print("Acurácia: %f" % resumo_lr_treino.accuracy)
print("Precisão: %f" % resumo_lr_treino.precisionByLabel[1])
print("Recall: %f" % resumo_lr_treino.recallByLabel[1])
print("F1: %f" % resumo_lr_treino.fMeasureByLabel()[1])

Acurácia: 0.784901
Precisão: 0.770686
Recall: 0.812517
F1: 0.791049


> Todas as métricas estão próximas de 0.8, em relação aos dados de treino. Isso diz o quão bem o modelo se ajustou, pois é uma característica importante, mas também tem que se sair bem em relação aos de teste.

Para isso, é necessário analisar os dados através de uma **matriz de confusão**. No entanto, o Spark não traz no módulo de DataFrame a criação dessa matriz, sendo necessário calcular e extrair esses dados.

A previsão dos dados de teste está salvo na variável `previsoes_lr_teste`. Então, é através dela que os cálculos serão feito.

In [31]:
# Seleção (label verdadeiro e label predito) e visualização dos dados:
# É preciso analisar os dados reais que estão salvos na coluna 'label'
# .select() para ver a predição que o modelo classificou em relação a um cliente
previsoes_lr_teste.select('label', 'prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       1.0|
|    1|       0.0|
|    1|       0.0|
|    0|       1.0|
|    0|       1.0|
|    0|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       1.0|
|    1|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



- Primeira cláusula: queremos que a classe real seja igual a 1, ou seja, que cliente tenha cancelado o serviço;
- Segunda cláusula: usa & para separar as cláusulas, e dizer que o f.col, a seleção de uma coluna, vai ser a 'prediction'.

In [32]:
# Criação de cláusulas (verifica se a classificou corretamente)
previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
|    1|       1.0|
+-----+----------+
only showing top 5 rows



In [ ]:
# Apaga o .show() porque o interesse é na contagem
# Usa .count() no final
previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()

1256

In [33]:
# Repete o processo para as demais possibilidades e salva em variáveis
tp = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count() # true positive: acertou o que era
tn = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count() # true negative: acertou o que não era
fp = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count() # false positive: falou que era, mas não era
fn = previsoes_lr_teste.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count() # false negative: falou que não era, mas era
print(tp, tn,fp, fn)

1256 1179 400 307


In [54]:
# Criando uma função para utilizar em outros modelos
def calcula_mostra_matriz_confusao(df_transform_modelo, normalize=False, percentage=True):
  tp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
  tn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
  fp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
  fn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()

  valorP = 1
  valorN = 1

  if normalize:
    valorP = tp + fn
    valorN = fp + tn

  if percentage and normalize:
    valorP = valorP / 100
    valorN = valorN / 100

  print(' '*20, 'Previsto')
  print(' '*15, 'Churn', ' '*5 ,'Não-Churn')
  print(' '*4, 'Churn', ' '*6, int(tp/valorP), ' '*7, int(fn/valorP))
  print('Real')
  print(' '*4, 'Não-Churn', ' '*2, int(fp/valorN), ' '*7, int(tn/valorN))

In [ ]:
# sem normalizar, pois o interesse são os dados calculados
calcula_mostra_matriz_confusao(previsoes_lr_teste, normalize=False)

                     Previsto
                Churn       Não-Churn
     Churn        1256         307
Real
     Não-Churn    400         1179


> Por exemplo, existem 1256 dados que foram previstos com churn, e que realmente o eram (**true positive**). Já na coluna do lado, foi previsto 307 que não foram churn, mas, na verdade, eles eram churn, então o modelo errou (false positive).

Se observar a diagonal da matriz, tem o valor 307 e o 400, que são os valores que a matriz errou. Errou tanto em falar que era churn e que na verdade não era,e não-churn que na verdade era.

Essa métrica é muito importante para fazer a análise do modelo e criar uma referência para comparar com outros modelos. A partir disso é possível entender se a regressão logística é o modelo ideal para resolver o problema proposto, ou se é algum outro modelo.

In [ ]:
from utils import calcula_mostra_metricas

print(calcula_mostra_metricas(modelo_lr, previsoes_lr_teste, normalize=True))

                    Previsto
               Churn     Não-Churn
    Churn      80       19
Real
    Não-Churn  25       74

Acurácia: 0.7849014709963918
Precisão: 0.7706855791962175
Recall: 0.8125173082248685
F1: 0.7910488002156916



# Testando: **Árvore de Decisão**:

**O modelo:** As Árvores de decisão (_Decision Tree Classifier_) são métodos populares para tarefas de machine learning de classificação e regressão. As árvores de decisão são amplamente utiizadas por serem fáceis de interpretar, lidar com features categóricas e contínuas, podem ser aplicadas a problemas de classificação com múltiplas classes e não requerem padronização dos dados. Algoritmos de conjuntos de árvores, como _Random Forest_ e _Boosting_, estão entre os de melhor desempenho para tarefas de classificação e regressão.

Esse tipo de algoritmo trabalha pegando os dados fornecidos para ajuste e analisa cada feature, buscando criar ramificações e calculando a pureza de cada grupo. O algoritmo repete esse processo enquanto seleciona outros valores para as features e mesmo outras features, buscando purificar o máximo possível cada grupo até criar grupos contendo somente uma classe por ramo, chamado de folha (**essa folha é a classificação de fato**). Esse modelo realiza cálculos de ganho de informação, ou seja, calcula o quanto ele conseguiu purificar determinado ramo tomando a decisão de olhar para determinada feature e determinado valor.

Graficamente, a árvore de decisão pode ser representada de forma que cada uma das decisões tomadas no processo possam ser visualizadas. Seus elementos principais são os nós, ramos e folhas. A estrutura da árvore se inicia com um nó inicial, também chamado de raiz. A partir dela são traçadas ramificações, que geram novos nós e o processo se repete para os nós subsequentes até que chegue a uma folha, que se trata de um nó especial que tem a informação da resposta, sendo ela uma categoria ou um valor previsto. Cada ramo representa uma tomada de decisão a partir de um valor ou de uma categoria das variáveis explicativas, dividindo o conjunto de dados em nós que apresentam dados com características cada vez mais similares entre si.


**Principais critérios para divisão dos nós:**
- Índice Gini: mede grau de heterogeneidade dos dados. O objetivo dele é medir a frequência de um elemento aleatório de um nó ser rotulado de maneira incorreta. Se o índice Gini for igual a 0, isso indica que o nó é puro. No entanto, se o valor dele se aproxima mais do valor 1, o nó é impuro.
- Entropia: mede desordem dos dados de um nó por meio da variável classificadora.

Algumas vantagens das árvores de decisão:

- não são sensíveis à escala dos dados, evitando a necessidade de modificações;
- são fáceis de explicar: é possível gerar essas árvores que facilitam o entendimento das decisões tomadas pelo algoritmo.

**Objetivo**: utilizar esse algoritmo para identificar pessoas que pretendem cancelar ou não o serviço.

## **Ajuste e Previsão usando modelo Decision Tree**

In [34]:
from pyspark.ml.classification import DecisionTreeClassifier

In [35]:
# esse algoritmo serve aleatoriedade para trabalhar e tomar decisões, portante é importante passar a SEED para garantir os mesmos resultados
dtc = DecisionTreeClassifier(seed=SEED)

In [36]:
# ajustando o modelo aos dados de treino
modelo_dtc = dtc.fit(treino)

In [37]:
# previsões
previsoes_dtc_treino = modelo_dtc.transform(treino)

In [ ]:
# verificando as previsões
previsoes_dtc_treino.show(15)

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [22.0,3.0]|         [0.88,0.12]|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,

**Aqui tem as mesmas 5 colunas que existe na regressão logística**

  - "features", a qual informa as features armazenando as características do cliente;
  - "label" que corresponde à classificação real dos dados informando se o cliente cancelou ou não o serviço;
  - "rawPrediction";
  - "probability";
  - "prediction", a qual corresponde à decisão final dos ramos da árvore em relação à determinado cliente.

### **Métricas para avaliar o modelo:**

Utilizando outra ferramenta - Multiclass classification evaluator, que é um avaliador de classificação de mais de duas classes. Essa ferramenta funciona bem para o problema do projeto que é uma classificação binária.

In [41]:
# importa a classe
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [39]:
# instancia o avaliador
evaluator = MulticlassClassificationEvaluator()

In [ ]:
# vai precisar das classificações que o modelo fez
# na variável previsoes_dtc_treino tem os dados reais e preditos
# segundo parâmetro: qual a métrica de interesse. Nesse caso foi a acurácia
# o seg parâmetro se passa no formato de dicionário
evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: 'accuracy'})

0.7917013599777962

In [ ]:
# outras métricas disponíveis:
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Acurácia: 0.791701
Precisão: 0.805090
Recall: 0.770978
F1: 0.787664


> Os resultados demonstram uma boa aderencia aos dados de treino (~80%).

**Mas se ajusta bem tb com os dados de teste?** // o modelo generaliza bem?

In [40]:
# previsões com os dados de teste
previsoes_dtc_teste = modelo_dtc.transform(teste)

In [ ]:
previsoes_dtc_teste.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [62.0,128.0]|[0.32631578947368...|       1.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    1| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [51.0,141.0]| [0.265625,0.734375]|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[331.0,1951.0]|[0.14504820333041...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [239.0,205.0]|[0.53828828828828...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [63.0,118.0]|[0.34806629834254...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[2056.0,334.0]|[0.86025104602510...|       0.0|
|(24,[0,1,2,

In [ ]:
# mesma estratégia para calcular as métricas do modelo
evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: 'accuracy'})

0.7714831317632082

> Próximo aos dados de treino

In [ ]:
# Relatório completo de todas as métricas e facilitar a interpretação
print('Decision Tree Classifier')
print("="*40)
print("Dados de Treino")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_dtc_treino, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print("")
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_dtc_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier
Dados de Treino
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        2784         827
Real
     Não-Churn    674         2921
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.791701
Precisão: 0.805090
Recall: 0.770978
F1: 0.787664

Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1181         382
Real
     Não-Churn    336         1243
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.771483
Precisão: 0.778510
Recall: 0.755598
F1: 0.766883


> **Importante**: Análise da matriz de confusão dos testes: erros da diagonal -> nos dados de treino ele previu que 827 pessoas não cancelariam o serviço e na vdd elas cancelaram. Um erro parecido acontece para a previsão de que 674 pessoas cancelariam o serviço, mas na verdade eles não iriam cancelar o serviço. É um número expressivo considerando o volume dos dados  disponível.

Para os dados de teste, o modelo previu 382 pessoas que não cancelariam o serviço e cancelaram (falso negativo). Isso é problemático, pois em teoria se o setor de marketing oferecer serviços para esses clientes seria uma perda de recurso. **A taxa de falso negativo precisa ser diminuída ao máximo.**

**Conclusão**: mais algoritmos precisam ser explorados para melhorar esses resultados.

In [ ]:
# Usando a função externa
from utils import calcula_mostra_metricas_evaluate
print(calcula_mostra_metricas_evaluate(previsoes_lr_teste, normalize=False))

# Testando: **Random Forest**:

Foram construídos dois modelos que conseguem classificar os clientes que vão ou não cancelar o serviço de assinatura. Porém, considerando os resultados, ainda parece ser preciso buscar um modelo que resolva esse problema com mais assertividade.

As árvores de decisão possuem uma característica que as impedem de serem consideradas como ferramenta ideal devido a sua imprecisão. Isso quer dizer que elas funcionam muito bem com os dados utilizados para criá-las, mas não tão bem para realizar a classificação de novas amostras. A floresta aleatória (random forest) busca resolver esse problema de _overfitting_ (sobreajuste).

O **Random Forest Classifier** é considerado uma evolução da árvore de decisão e se baseia na utilização de diversas árvores de decisão para encontrar o resultado.

Para realizar a previsão, o algoritmo cria diversas árvores de decisão no conjunto de dados e é realizada a predição para cada uma delas. Internamente é feita uma “votação” para analisar qual predição tem maior ocorrência e, então, essa predição torna-se a resposta final. Se fosse utilizada a mesma base de dados na criação de todas as árvores de decisão do Random Forest, as respostas de cada uma das árvores seriam iguais e o resultado da votação seria idêntico a realizar um único modelo de árvore de decisão.

Para evitar esse problema, é utilizada uma técnica chamada **bootstrapping**. Ela consiste em fazer amostragens com reposição do conjunto de dados original e cada uma delas será usada para uma árvore de decisão diferente. A amostragem com reposição significa que, ao sortearmos um elemento, isso não nos impede que ele mesmo apareça em sorteios futuros. Assim, as árvores terão resultados distintos, uma vez que são treinadas com conjuntos de dados diferentes.

Esse algoritmo pega os dados de treino e quebra conforme a quantidade de features que vão ser utilizados. Por exemplo, se for fornecido 5 features para serem analisados na primeira árvore, ela seguirá o processo natural de construção, buscando a pureza do ramo e o ganho de informação, tudo isso pensando nas features. Depois, na segunda árvore de decisão é treinado as mesmas ou outras features, com grupos de quantidade diferentes de features a serem exploraras. Cada árvore será responsável por determinadas features e a partir dos dados de treino fornecido, vai se ajustar e tomar a decisão final.

Uma desvantagem seria a maior necessidade de processamento, por isso é necessário avaliar se é uma opção que realmente vai resolver o problema de forma eficiente.

- Ao utilizar o modelo Random Forest, é possível escolher a quantidade de árvores de decisão a serem criadas. No scikit learn essa quantidade é constrolada através do parâmetro n_estimators;
- O modelo irá criar um conjunto de dados para cada árvore a partir do método bootstrapping na base de dados original, resultando em um resultado distinto para cada uma das árvores;
- No fim será feita uma votação entre os resultados das árvores e a classe predita na maioria das árvores é escolhida como a classificação do modelo Random Forest.



In [35]:
# importa o módulo
from pyspark.ml.classification import RandomForestClassifier

In [42]:
# tem fator de aleatoriedade dentro dele e por isso add a seed
rfc = RandomForestClassifier(seed=SEED)

In [43]:
# ajusta o modelo com os dados de treino
modelo_rfc = rfc.fit(treino)

In [44]:
# previsões do modelo com os dados de treino
previsoes_rfc_treino = modelo_rfc.transform(treino)

In [ ]:
# df composto das mesmas 5 colunas que o lr e decision tree
previsoes_rfc_treino.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[15.0052773466704...|[0.75026386733352...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[16.9295040273249...|[0.84647520136624...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[9.13052909106814...|[0.45652645455340...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.13052909106814...|[0.45652645455340...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[8.59288938528764...|[0.42964446926438...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[5.59647122885698...|[0.27982356144284...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.33276328267787...|[0.46663816413389...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[5.21616013157118...|[0.26080800657855...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[5.45640255581361...|[0.27282012779068...|       1.0|
|(24,[0,1,2,3,4,

### **Métricas para avaliar o modelo:**

Para calcular as métricas é preciso fazer a transformação dos dados para os de teste.

In [45]:
previsoes_rfc_teste = modelo_rfc.transform(teste)

In [ ]:
# visualizando as previsões de teste
previsoes_rfc_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[16.7433871675615...|[0.83716935837807...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[7.27313214599648...|[0.36365660729982...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[7.46885072161585...|[0.37344253608079...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[9.33276328267787...|[0.46663816413389...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.79829004739264...|[0.38991450236963...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.13263407834549...|[0.35663170391727...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[4.45872635511159...|[0.22293631775557...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[7.84691519125130...|[0.39234575956256...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[9.94796150783366...|[0.49739807539168...|       1.0|
|(24,[0,1,2,3,4,

In [ ]:
# Sumário completo com as métricas do modelo
print('Random Forest Classifier')
print("="*40)
print("Dados de Treino")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_rfc_treino, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print("")
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_rfc_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier
Dados de Treino
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        2950         661
Real
     Não-Churn    884         2711
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.785595
Precisão: 0.769431
Recall: 0.816948
F1: 0.792478

Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1257         306
Real
     Não-Churn    416         1163
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.770210
Precisão: 0.751345
Recall: 0.804223
F1: 0.776885


Comparando com as métricas obtidas pelo modelo da Decision Tree.

- **Falsos negativos**:

A **Decision Tree Classifier estava errando bastante nos falsos negativos** dos _dados de treino_, sendo _827 erros_. Já a **Random Forest Classifier** teve _661 erros_, isso indica que ela teve um ajuste melhor aos dados de treino e performou melhor nessa métrica.

- **Precisão**:

A **precisão**, em relação aos dados de treino, da **árvore de decisão foi superior a Random Forest Classifier**.

Em relação aos dados de teste e quão bem o modelo consegue generalizar: está parecido com os dados de teste. A Random Forest errou mais para falsos positivos quando comparado a Decision Tree.

- **Dados de Teste**:

Nenhum dos nossos modelos conheciam anteriormente os dados de teste. Dessa forma, a árvore de decisão pontuou que 382 clientes não entrariam em churn, mas entraram, portanto, esse foi seu número de erros. Enquanto isso, a floresta errou um pouco menos, 306. Porém, a árvore de decisão classificou que 336 clientes cancelariam o serviço, mas não cancelaram. Já a floresta 416 clientes, portanto nessa métrica, apresentou mais erros.

 > **Conclusão**: os dois **algoritmos de arvores estão perfomando bem. Seguindo o pipeline de ajuste é necessário realizar a otimização dos hiperparametros**, uma vez que até o momento foram utilizados os parâmetros default.

In [ ]:
from utils import compara_metricas_modelos

print(compara_metricas_modelos({'Logistic Regression': previsoes_lr_teste,
                                'DecisionTreeClassifier': previsoes_dtc_teste,
                                'RandomForestClassifier': previsoes_rfc_teste}))

# **Otimização de Hiperparâmetros**

Cada um dos modelos de machine learning possui um comportamento distinto para prever os resultados utilizando métodos matemáticos e computacionais. O comportamento de um modelo dependerá de constantes, parâmetros ou características para que a fórmula matemática ou o procedimento computacional se comporte de maneira diferente.

Esses argumentos que controlam o comportamento de um modelo de machine learning são chamados de hiperparâmetros. Ao alterar o valor desses parâmetros, modificamos também o desempenho do modelo, uma vez que, para cada conjunto de dados diferentes, é necessário um ajuste diferente dos hiperparâmetros para um modelo ser melhor adaptado a esse conjunto.

Hiperparâmetros Árvore de Decisão:

**maxDepth**: profundidade máxima que a árvore pode alcançar;

Outro hiperparâmetro que pode ser controlado na árvore de decisão é o minInstancesPerNode, que está relacionado à quantidade mínima de amostras dos dados de treinamento para cada folha. Isso significa que, se há um nó na árvore com a quantidade de registros definida em minInstancesPerNode, este nó não pode ser dividido em outros nós. c

**Validação Cruzada**: é um procedimento de reamostragem dos dados, variando os grupos de dados, que serão ajustados e a performance média do modelo. Entende melhor o comportamento do algoritmo com os dados. É uma etapa para diminuir o efeito da aleatoriedade ao selecionar um grupo de dados. Essa técnica corresponde a umas das ferramentas presentes no pacote de _tuning_ do PySpark. O módulo de **tuning** se refere aos hiperparâmetros.

Na etapa de tuning existem 2 ferramentas: **a de variar os hiperparâmetros**; e a de **criar a validação cruzada** na ferramenta.

In [38]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [50]:
# Otimização de hiperparâmetros - Arvore de Decisão
# dtc = DecisionTreeClassifier(seed=SEED)

In [52]:
# grid: etapa de seleção dos hiperparâmetros e valores que serão explorados
# addGrid recebe os [valores que serão explorados]
# testando profundidade [2, 5, 10]
grid = ParamGridBuilder()\
        .addGrid(dtc.maxDepth, [2, 5, 10])

In [53]:
# adicionando hiperparametro maxBins
grid = ParamGridBuilder()\
        .addGrid(dtc.maxDepth, [2, 5, 10])\
        .addGrid(dtc.maxBins, [10, 32, 45])

> Isso quer dizer que serão testadas todas as combinações de `maxDepth` e `maxBins` em algoritmos, novamente, buscando encontrar o melhor ajuste para os dados.

In [54]:
# add .build(), para que de fato seja construído o ParamGrid
grid = ParamGridBuilder()\
        .addGrid(dtc.maxDepth, [2, 5, 10])\
        .addGrid(dtc.maxBins, [10, 32, 45])\
        .build()

In [55]:
# construção de um avaliador
# como será avaliado com qual hiperparâmetro determinado algoritmo funciona melhor?
evaluator = MulticlassClassificationEvaluator()

**Agora, é possível utilizar o CrossValidator de fato, passando essas informações para que ele faça a validação cruzada e busque pelo hiperparâmetro ideal.**

Construção do CrossValidator:
- `estimator`: estimador que será utilizado, por exemplo, RandomForestClassifier ou DecisionTreeClassifier.

- `estimatorParamMaps`: parâmetros e valores desses parâmetros que deverão ser utilizados ao realizar a validação cruzada.

- `evaluator`: define o objeto responsável por avaliar os modelos.

- `numFolds`: número de dobras, ou seja, quantas partes do conjunto de dados deverão ser utilizados para treino e teste a cada iteração da validação cruzada.

In [57]:
# construção do CrossValidator
# estimator: qual algoritmo está sendo avaliado? dtc (DecisionTreeClassifier)
# estimatorParamMaps: responsável pelo mapeamento dos hiperparâmetros escolhidos
# evaluator: avalia o desempenho do modelo
# Dois parametros ligados ao funcionamento da validação cruzada:
# numFolds: responsavel pela quebra dos dados;
# seed: reproduzir aleatoriedade
dtc_cv = CrossValidator(
    estimator=dtc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=SEED
)

Ao realizar a validação cruzada é possível definir o número de folds (dobras) que será utilizado para treino e teste. Por exemplo, se o número de dobras for igual a 3, isso significa que o conjunto de dados será dividido em 3 partes. Logo, o modelo será treinado na parte 2 e 3 e testado na parte 1.

Na próxima iteração será treinado na parte 1 e 3 e testado na parte 2 e, assim, sucessivamente até que todas as partes tenham sido utilizadas para teste. Dessa forma, a média das acurácias obtidas nos fornecerá uma visão mais geral de como o modelo se comporta para dados que o modelo não conhece.

In [58]:
# Usando dtc_cv (DecisionTreeClassifier CrossValidator) para ajuste e previsão dos dados
modelo_dtc_cv = dtc_cv.fit(treino)

> Nesse processo são avaliados diversos algortimos, variando combinações de hiperparâmetros, conforme determinado no grid, e busca os melhores hiperparâmetros para o conjunto de dados.

In [59]:
# previsões para os dados de teste
previsoes_dtc_cv_teste = modelo_dtc_cv.transform(teste)

In [62]:
print('Decision Tree Classifier - Tuning')
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_dtc_cv_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier - Tuning
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1319         244
Real
     Não-Churn    430         1149
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.785487
Precisão: 0.754145
Recall: 0.843890
F1: 0.796498


Para os dados de teste, na matriz de confusão é possível observar 244 falsos positivos ("Não-Churn"), classificados como se o cliente não fosse cancelar o serviço, sendo que na verdade cancelou.

Portanto, existiram **244 erros em situações que devia ter sido priorizado o atendimento a esses clientes**; como não ocorreu, existiu uma perda pois o serviço foi cancelado.

**Essa é uma métrica que deve ser otimizada**. É possível comparar esse resultado obtido com a dtc original, que teve os valores padrões.

Para a métrica analisada anteriormente, referente a **falsos negativos, o número passou de 382 para 244**, ou seja, foi possível **reduzir essa métrica considerada importante**, uma vez que evita-se perder clientes (alinhado com o objetivo inicial do projeto).

Vendo as outras métricas, como **acurácia, precisão, recall e F1 Score é possível observar melhorias** para quase todas, com **exceção da precisão**: com os parâmetros padrão, onde era 0.77 e passou para 0.75.

> Devemos nos perguntar: **o mais importante para a nossa área de negócios é acertar todos os casos e classificar muito bem todas as situações (se o cliente cancela ou não), ou algum dos dois tem maior relevância?**

**_Nesse caso, talvez a métrica da precisão não seja tão importante quanto acertar os falsos negativos._**

Dessa forma é necessário refletir e entender sobre a área de negócios em que se está trabalhando, para, então, tomar a decisão final.

# **Otimização de Hiperparâmetros**: Random Forest

In [45]:
# instanciar o algortimo
rfc = RandomForestClassifier(seed=SEED)

In [49]:
# definir o espaço que será encontrado o melhor hiperparametro
# numTrees: caracteristica do random forest: número de árvores
grid = ParamGridBuilder()\
        .addGrid(rfc.maxDepth, [2, 5, 10])\
        .addGrid(rfc.maxBins, [10, 32, 45])\
        .addGrid(rfc.numTrees, [10, 20, 50]).build()

In [50]:
evaluator = MulticlassClassificationEvaluator()

In [43]:
rfc_cv = CrossValidator(
    estimator=rfc,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=SEED
)

In [51]:
modelo_rfc_cv = rfc_cv.fit(treino)

In [52]:
previsoes_rfc_cv_teste = modelo_rfc_cv.transform(teste)

In [55]:
print('Random Forest Classifier - Tuning')
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
calcula_mostra_matriz_confusao(previsoes_rfc_cv_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier - Tuning
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1333         230
Real
     Não-Churn    337         1242
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.819542
Precisão: 0.798204
Recall: 0.852847
F1: 0.824621


#### **O modelo final:**

O que é necessário para treinar o modelo final? _Saber quais foram os melhores hiperparâmetros que a busca encontrou_.







In [56]:
melhor_modelo_rfc_cv = modelo_rfc_cv.bestModel

In [57]:
# pegando os melhores hiperparametros
print(melhor_modelo_rfc_cv.getMaxDepth())
print(melhor_modelo_rfc_cv.getMaxBins())
print(melhor_modelo_rfc_cv.getNumTrees)

10
45
50


> Valores encontrados: **10, 45 e 50**.

In [58]:
# usando os hiperparametros encontrados para criar um novo modelo
rfc_tunning = RandomForestClassifier(maxDepth=10, maxBins=45, numTrees=50, seed=SEED)

In [59]:
# etapa de ajuste aos dados de treino
modelo_rfc_tunning = rfc_tunning.fit(dataset_prep)

> **Como esse é o nosso modelo final, o foco é utilizar o máximo de dados possível. Por isso, se passa todos os dados, no caso o dataset chamado dataset_prep.**

#### Como esse modelo vai funcionar quando passar para a produção?

**É necessário passar as informações de um cliente novo para esse modelo.**

Com esse objetivo, coloca a variável X em uma nova célula para lembrar das colunas que existiam. No caso, as colunas como 'Mais65anos', 'MesesDeContrato' e'MesesCobrados'.

**São essas as características que se colhe do contrato do cliente para decidir se vai cancelar o serviço ou não.**

Analisando essas colunas, é possível criar um novo cliente. O objetivo é descobrir se essa pessoa específica vai cancelar o serviço ou não.



In [60]:
novo_cliente = [{
    'Mais65anos': 0,
    'MesesDeContrato': 1,
    'MesesCobrados': 45.30540797610398,
    'Conjuge': 0,
    'Dependentes': 0,
    'TelefoneFixo': 0,
    'MaisDeUmaLinhaTelefonica': 0,
    'SegurancaOnline': 0,
    'BackupOnline': 0,
    'SeguroDispositivo': 0,
    'SuporteTecnico': 0,
    'TVaCabo': 1,
    'StreamingFilmes': 1,
    'ContaCorreio': 1,
    'Internet_DSL': 1,
    'Internet_FibraOptica': 0,
    'Internet_Nao': 0,
    'TipoContrato_Mensalmente': 1,
    'TipoContrato_UmAno': 0,
    'TipoContrato_DoisAnos': 0,
    'MetodoPagamento_DebitoEmConta': 0,
    'MetodoPagamento_CartaoCredito': 0,
    'MetodoPagamento_BoletoEletronico': 1,
    'MetodoPagamento_Boleto': 0
}]

> **Essa é uma lista que tem um dicionário, onde as chaves são os nomes das colunas - as características do cliente e do contrato dele. E o valor são os valores respectivos.**

Por exemplo, esse cliente tem o valor 0 na chave 'Mais65anos'. Isso significa que essa pessoa não tem mais de 65 anos. Também pode-se notar que tem apenas 1 mês de contrato, 45 meses cobrados e não possui dependentes. São essas diversas características que o cliente apresenta.

Agora, é preciso **transformar esses dados de maneira que o modelo consiga entendê-los** (igual aos dados brutos no início da nossa análise).


In [61]:
# cria um DataFrame Spark, passando novo_cliente
novo_cliente = spark.createDataFrame(novo_cliente)
novo_cliente.show()

+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+-----------------+---------------+----------------------+--------------------------------+-----------------------------+-----------------------------+---------------+-----------------+---------------+--------------+-------+------------+---------------------+------------------------+------------------+
|BackupOnline|Conjuge|ContaCorreio|Dependentes|Internet_DSL|Internet_FibraOptica|Internet_Nao|Mais65anos|MaisDeUmaLinhaTelefonica|    MesesCobrados|MesesDeContrato|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|SegurancaOnline|SeguroDispositivo|StreamingFilmes|SuporteTecnico|TVaCabo|TelefoneFixo|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|
+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+----

In [62]:
# passa os dados para a característica do DataFrame Spark
# fazer a transformação dos dados -> para um vetor
assembler = VectorAssembler(inputCols = X, outputCol = 'features')

In [63]:
# novo_cliente_prep é para o novo cliente vetorizado que vai ser igual a assembler.transform()
novo_cliente_prep = assembler.transform(novo_cliente).select('features')

In [64]:
# analisando o resultado do novo_cliente_prep
novo_cliente_prep.show(truncate=False)

+----------------------------------------------------------------------------+
|features                                                                    |
+----------------------------------------------------------------------------+
|(24,[1,2,11,12,13,14,17,22],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])|
+----------------------------------------------------------------------------+



In [65]:
# faz a previsão
modelo_rfc_tunning.transform(novo_cliente_prep).show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(24,[1,2,11,12,13...|[5.16473071594386...|[0.10329461431887...|       1.0|
+--------------------+--------------------+--------------------+----------+



> Tabela com 4 colunas: "features" (as características/problemas), "rawPrediction", "probability" e "prediction".

**A coluna "prediction" contém a informação de interesse. O valor 1.0 significa que o modelo prevê que esse cliente vai cancelar o serviço. _Isso significa que esse seria um cliente que o time de marketing deveria priorizar e oferecer promoções para tentar retê-lo_.**

Isso também significa que o modelo está funcionando, pois faz previsões para clientes que ainda não conhecia com base no histórico que se ajustou.

**Próximo passo:** colocar o modelo em produção.


In [66]:
# Otimizando Logistic Regression
lr = LogisticRegression()

In [67]:
# definindo hiperparametros para explorar
grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.elasticNetParam, [0.0, 1.0])\
    .build()

In [68]:
# treinar o novo modelo e avaliar os resultados
lr_cv = CrossValidator(
    estimator=lr,
    estimatorParamMaps=grid,
    evaluator=evaluator,
    numFolds=3,
    seed=SEED
)

In [69]:
modelo_lg_cv = lr_cv.fit(treino)